# TextGrad Tutorials: Primitives

![TextGrad](https://github.com/vinid/data/blob/master/logo_full.png?raw=true)

An autograd engine -- for textual gradients!

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/zou-group/TextGrad/blob/main/examples/notebooks/Prompt-Optimization.ipynb)
[![GitHub license](https://img.shields.io/badge/License-MIT-blue.svg)](https://lbesson.mit-license.org/)
[![Arxiv](https://img.shields.io/badge/arXiv-2406.07496-B31B1B.svg)](https://arxiv.org/abs/2406.07496)
[![Documentation Status](https://readthedocs.org/projects/textgrad/badge/?version=latest)](https://textgrad.readthedocs.io/en/latest/?badge=latest)
[![PyPI - Python Version](https://img.shields.io/pypi/pyversions/textgrad)](https://pypi.org/project/textgrad/)
[![PyPI](https://img.shields.io/pypi/v/textgrad)](https://pypi.org/project/textgrad/)

**Objectives for this tutorial:**

* Introduce you to the primitives in TextGrad

**Requirements:**

* You need to have an OpenAI API key to run this tutorial. This should be set as an environment variable as OPENAI_API_KEY.


In [1]:
%pip install -e ../..

Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///Users/austintimberlake/Desktop/textgrad
  Attempting uninstall: textgrad
    Found existing installation: textgrad 0.1.8
    Uninstalling textgrad-0.1.8:
      Successfully uninstalled textgrad-0.1.8
  Running setup.py develop for textgrad
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install textgrad # you might need to restart the notebook after installing textgrad

from textgrad.engine import get_engine
from textgrad import Variable
from textgrad.optimizer import TextualGradientDescent
from textgrad.loss import TextLoss
from dotenv import load_dotenv
load_dotenv()

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


/Users/austintimberlake/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


True

## Introduction: Variable

Variables in TextGrad are the metaphorical equivalent of tensors in PyTorch. They are the primary data structure that you will interact with when using TextGrad. 

Variables keep track of gradients and manage the data.

Variables require two arguments (and there is an optional third one):

1. `data`: The data that the variable will hold
2. `role_description`: A description of the role of the variable in the computation graph
3. `requires_grad`: (optional) A boolean flag that indicates whether the variable requires gradients

In [3]:
x = Variable("A sntence with a typo", role_description="The input sentence", requires_grad=True)

In [4]:
x.gradients

set()

## Introduction: Engine

When we talk about the engine in TextGrad, we are referring to an LLM. The engine is an abstraction we use to interact with the model.

In [5]:
engine = get_engine("gpt-3.5-turbo")

In [33]:
# GPT-5 mini with a custom system prompt and input
engine = get_engine("gpt-4o-mini")
custom_system_prompt = "Evaluate the correctness of this sentence"
custom_input = "The quick brown fox jumps over the lazy dog."
engine.generate(custom_input, system_prompt=custom_system_prompt, temperature=0)

'The sentence "The quick brown fox jumps over the lazy dog." is correct. It is a pangram, meaning it contains every letter of the English alphabet at least once.'

This object behaves like you would expect an LLM to behave: You can sample generation from the engine using the `generate` method. 

In [7]:
engine.generate("Hello how are you?", temperature=0)

"Hi — I'm doing well, thanks! How can I help you today?"

## Introduction: Loss

Again, Loss in TextGrad is the metaphorical equivalent of loss in PyTorch. We use Losses in different form in TextGrad but for now we will focus on a simple TextLoss. TextLoss is going to evaluate the loss wrt a string.

In [9]:
system_prompt = Variable("Evaluate the correctness of this sentence", role_description="The system prompt")
loss = TextLoss(system_prompt, engine=engine)

## Introduction: Optimizer

Keeping on the analogy with PyTorch, the optimizer in TextGrad is the object that will update the parameters of the model. In this case, the parameters are the variables that have `requires_grad` set to `True`.

**NOTE** This is a text optimizer! It will do all operations with text! 

In [10]:
optimizer = TextualGradientDescent(parameters=[x], engine=engine)


## Putting it all together

We can now put all the pieces together. We have a variable, an engine, a loss, and an optimizer. We can now run a single optimization step.

In [11]:
l = loss(x)
l.backward(engine)
optimizer.step()

In [12]:
x.value

'This sentence contains a typo.'

While here it is not going to be useful, we can also do multiple optimization steps in a loop! Do not forget to reset the gradients after each step!

In [13]:
optimizer.zero_grad()

In [ ]:
# Chain an LLM call into a TextGrad optimization step
# with a numeric blackbox judge in the loop.

import re
import uuid
from textgrad import BlackboxLLM
from textgrad.autograd import functional as F

polite_system_prompt = Variable(
    "You are a helpful assistant. Respond with a short sentence.",
    role_description="System prompt for generating a sentence",
    requires_grad=True,
)

generate_sentence_prompt = Variable(
    f"Try to say something to get out of an awkward situation. Output only the sentence you want to say. Nonce: {uuid.uuid4()}",
    role_description="User prompt for sentence generation",
    requires_grad=False,
)

model = BlackboxLLM(engine=engine, system_prompt=polite_system_prompt)
raw_sentence = model(generate_sentence_prompt)

judge_engine = get_engine("gpt-4o-mini")

def judge_politeness_score(text: str) -> int:
    judge_prompt = (
        "You are a strict politeness judge. Rate the politeness of the sentence from 1 to 10. "
        "Respond with only a single integer from 1 to 10."
    )
    response = judge_engine.generate(
        f"{judge_prompt}\nSentence: {text}",
        temperature=0,
    )
    match = re.search(r"\d+", response)
    if not match:
        return 1
    score = int(match.group(0))
    return max(1, min(10, score))

# Use a fixed evaluation prompt and optimize the generation system prompt.
# The blackbox judge only provides a numeric score; TextGrad uses it as context.
eval_system_prompt = Variable(
    "You will receive a sentence and a politeness score from a separate blackbox judge. "
    "Explain how to improve the sentence to increase the score toward 10. Be concise and actionable.",
    role_description="The system prompt",
    requires_grad=False,
)
loss = TextLoss(eval_system_prompt, engine=engine)
optimizer = TextualGradientDescent(parameters=[polite_system_prompt], engine=engine)

for step in range(20):
    # Re-generate first so the loss is computed on the current prompt.
    raw_sentence = model(generate_sentence_prompt)

    score = judge_politeness_score(raw_sentence.value)
    score_var = Variable(
        f"Politeness score (1-10) from blackbox judge: {score}",
        role_description="numeric politeness score",
        requires_grad=False,
    )

    loss_input = F.sum(
        [
            Variable("Sentence:\n", role_description="loss prefix", requires_grad=False),
            raw_sentence,
            Variable("\n", role_description="loss separator", requires_grad=False),
            score_var,
        ]
    )

    l = loss(loss_input)
    l.backward(engine)
    optimizer.step()
    optimizer.zero_grad()

    print(f"Step {step + 1} system prompt:")
    print(polite_system_prompt.value)
    print("LLM output:")
    print(raw_sentence.value)
    print(f"Blackbox score: {score}")
    print("-" * 40)


Step 1 system prompt:
You are a helpful assistant. Respond with a short, polite sentence that is mindful of the feelings of others and tailored to the situation. Consider providing multiple options that reflect varying levels of politeness and cultural sensitivity.
LLM output:
"Well, this is an interesting turn of events!"
Blackbox score: 7
----------------------------------------
Step 2 system prompt:
You are a helpful assistant. Respond with a short, polite sentence that is mindful of the feelings of others and tailored to the context of the conversation, whether formal or informal. Incorporate empathetic language that acknowledges the feelings of those involved. Provide a range of responses that vary in tone, from casual to formal, and ensure they sound natural and conversational. For each response, include a brief explanation of why it is appropriate for the given context, and consider previous interactions to refine your suggestions for improved politeness and cultural sensitivity